In [ ]:
# llama-cpp-python macos setup guide https://github.com/abetlen/llama-cpp-python/blob/main/docs/install/macos.md
# llama.cpp embeddings https://github.com/ggerganov/llama.cpp/issues/899

In [54]:
from llama_cpp import Llama
llm = Llama(model_path="models/nous-capybara-34b.Q4_K_M.gguf", embedding=True, verbose=False)

llama_model_loader: loaded meta data with 22 key-value pairs and 543 tensors from models/nous-capybara-34b.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  7168, 64000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_K     [  7168,  7168,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_K     [  7168,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  7168,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_K     [  7168,  7168,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_K     [  7168, 20480,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q4_K     [  7168, 20480,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weight q6_K     [ 20480,  7168,     1

In [ ]:
# output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)

In [55]:
import os
import json
from scipy.spatial.distance import cosine, pdist, cdist

directory = 'demo_articles'

doc_embeddings = []
chunks = []
fnames = []

for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'r') as file:
            text_content = file.read()
            paragraphs = text_content.split('\n\n')  
            num_paras = len(paragraphs)

            for i in range(0, num_paras, 2):
                start = i
                end = min(i+3, num_paras)
                text = ' '.join(paragraphs[start:end])
                chunks.append(text)
                if text.strip():
                    fnames.append(filename)
                    embedding = llm.embed(text)  
                    doc_embeddings.append(embedding)
                    
fnames

['Fava_claude.txt',
 'Fava_claude.txt',
 'Fava_claude.txt',
 'Haringsma_claude.txt',
 'Haringsma_claude.txt',
 'Haringsma_claude.txt',
 'Gastpar_claude.txt',
 'Gastpar_claude.txt',
 'Gastpar_claude.txt',
 'Wildcard.txt',
 'Wildcard.txt',
 'Wildcard.txt',
 'Wildcard.txt',
 'Wildcard.txt']

In [27]:
chunks

['"Eszopiclone co-administered with fluoxetine in patients with insomnia coexisting with major depressive disorder" is a study that was published in the \'Biol Psychiatry\' journal and authored by Fava. The article, identified with PubMed ID 16581036, explores the effects of combining Eszopiclone (a sleep aid) with Fluoxetine (an antidepressant) in treating patients who have both insomnia and major depressive disorder. The disease being studied is major depressive disorder with coexisting insomnia. The study population consisted of 545 patients randomized into two groups - 275 received fluoxetine plus eszopiclone, and 270 received fluoxetine plus placebo. There were 3 main therapies used in the study - fluoxetine, eszopiclone, and placebo. The fluoxetine dose was 20-40 mg daily. The eszopiclone dose was 3 mg at night. Both fluoxetine and eszopiclone were given for 8 weeks. Placebo was also given at night for 8 weeks.',
 'There were 3 main therapies used in the study - fluoxetine, eszop

In [71]:
prompt = "Insomnia coexisting with depression"
prompt_vec = llm.embed(prompt)

In [72]:
prompt

'Insomnia coexisting with depression'

In [73]:
import torch
from torch.nn.functional import cosine_similarity

# similarities = []
prompt_vec = torch.tensor(prompt_vec)

top_matches = []
for i, emb in enumerate(doc_embeddings):
    embt = torch.tensor(emb)
    cos_sim = cosine_similarity(prompt_vec.unsqueeze(0), embt.unsqueeze(0))
    top_matches.append((cos_sim, chunks[i]))
    
top_matches = sorted(top_matches, key=lambda x: x[0])
print(top_matches)

[(tensor([0.2736]), '"Eszopiclone co-administered with fluoxetine in patients with insomnia coexisting with major depressive disorder" is a study that was published in the \'Biol Psychiatry\' journal and authored by Fava. The article, identified with PubMed ID 16581036, explores the effects of combining Eszopiclone (a sleep aid) with Fluoxetine (an antidepressant) in treating patients who have both insomnia and major depressive disorder. The disease being studied is major depressive disorder with coexisting insomnia. The study population consisted of 545 patients randomized into two groups - 275 received fluoxetine plus eszopiclone, and 270 received fluoxetine plus placebo. There were 3 main therapies used in the study - fluoxetine, eszopiclone, and placebo. The fluoxetine dose was 20-40 mg daily. The eszopiclone dose was 3 mg at night. Both fluoxetine and eszopiclone were given for 8 weeks. Placebo was also given at night for 8 weeks.'), (tensor([0.2767]), 'The Bitcoin whitepaper, tit

In [8]:
sentences = ['Apples are my favorite fruit', 'Ive got the newest newspaper', 'The radio seems broken', 'Anybody seen my friend Josh?', 'There is an orchard nearby', 'Wishing you great harvest', 'We found some fruit in the fridge']
baseline = llm.embed('I love apples')
embeddings = []

for s in sentences:
    embeddings.append(llm.embed(s))
    
similarities = []
for i, emb in enumerate(embeddings):
    # emb_i = torch.mean(emb, dim=0) 
    cos_sim = cosine(baseline, emb)
    similarities.append((cos_sim.item(), sentences[i]))

similarities.sort(reverse=True)
print(similarities)

In [49]:
output = llm(f"Q: Condense this query into keywords: {prompt} A: ", max_tokens=300, stop=["Q:", "\n"], echo=True)
output

{'id': 'cmpl-e2e695b6-b3bb-402d-81a6-dcbf55263d53',
 'object': 'text_completion',
 'created': 1700656727,
 'model': 'models/openhermes-2.5-mistral-7b.Q4_K_M.gguf',
 'choices': [{'text': 'Q: Condense this query into keywords: In studies analyzing treatments for depression, with what frequency are headaches reported as a side effect? A: 1. Frequency 2. Depression treatments 3. Side effects 4. Headaches ',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 33, 'completion_tokens': 22, 'total_tokens': 55}}